In [1]:
!pip install pytorch-ignite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.4/272.4 kB 4.0 MB/s eta 0:00:00


In [2]:
import torch # we are going to use pytorch instead of numpy because it's much faster.
import torch.nn as nn
from ignite.contrib.metrics.regression import R2Score
import torch.optim as optim
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.datasets import make_spd_matrix
from scipy.optimize import minimize
from scipy.linalg import toeplitz
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse, r2_score as R2

In [3]:
device = torch.device("cpu")
dtype = torch.float64

In [4]:
class StandardScaler:
    def __init__(self):
        self.mean = None
        self.std = None

    def fit(self, data):
        """
        Compute the minimum and maximum value of the data for scaling.

        Args:
        - data (torch.Tensor): Input data tensor.
        """
        self.mean = torch.mean(data, dim=0, keepdim=True)
        self.std = torch.std(data, dim=0, keepdim=True) + 1e-10

    def transform(self, data):
        """
        Scale the data based on the computed minimum and maximum values.

        Args:
        - data (torch.Tensor): Input data tensor.

        Returns:
        - torch.Tensor: Scaled data tensor.
        """
        if self.mean is None or self.std is None:
            raise ValueError(
                "Scaler has not been fitted yet. Please call 'fit' with appropriate data."
            )

        scaled_data = (data - self.mean) / (self.std)
        return scaled_data

    def fit_transform(self, data):
        """
        Fit to data, then transform it.

        Args:
        - data (torch.Tensor): Input data tensor.

        Returns:
        - torch.Tensor: Scaled data tensor.
        """
        self.fit(data)
        return self.transform(data)

In [5]:
class MinMaxScaler:
    def __init__(self):
        self.min = None
        self.max = None

    def fit(self, data):
        """
        Compute the minimum and maximum value of the data for scaling.

        Args:
        - data (torch.Tensor): Input data tensor.
        """
        self.min = torch.min(data, dim=0, keepdim=True).values
        self.max = torch.max(data, dim=0, keepdim=True).values

    def transform(self, data):
        """
        Scale the data based on the computed minimum and maximum values.

        Args:
        - data (torch.Tensor): Input data tensor.

        Returns:
        - torch.Tensor: Scaled data tensor.
        """
        if self.min is None or self.max is None:
            raise ValueError(
                "Scaler has not been fitted yet. Please call 'fit' with appropriate data."
            )

        scaled_data = (data - self.min) / (self.max - self.min)
        return scaled_data

    def fit_transform(self, data):
        """
        Fit to data, then transform it.

        Args:
        - data (torch.Tensor): Input data tensor.

        Returns:
        - torch.Tensor: Scaled data tensor.
        """
        self.fit(data)
        return self.transform(data)

## PyTorch class that implements the method of SCAD regularization and variable selection (smoothly clipped absolute deviations) for linear models


In [26]:
import torch
import torch.nn as nn
import torch.optim as optim


class SCADRegressor(nn.Module):
    def __init__(self, input_size, alpha=1.0, a=3.7):
        """
        Initialize the SCADRegressor model.

        Args:
            input_size (int): Number of input features.
            alpha (float): Regularization strength.
            a (float): Threshold parameter for SCAD penalty.
        """
        super(SCADRegressor, self).__init__()
        self.input_size = input_size
        self.alpha = alpha
        self.a = a

        self.linear = nn.Linear(input_size, 1, bias=False)

    def forward(self, x):
        """
        Forward pass of the SCADRegressor model.

        Args:
            x (Tensor): Input data with shape (batch_size, input_size).

        Returns:
            Tensor: Predicted values with shape (batch_size, 1).
        """
        return self.linear(x)

    def loss(self, y_pred, y_true):
        """
        Compute the SCADRegressor loss function.

        Args:
            y_pred (Tensor): Predicted values with shape (batch_size, 1).
            y_true (Tensor): True target values with shape (batch_size, 1).

        Returns:
            Tensor: The SCADRegressor loss.
        """
        mse_loss = nn.MSELoss(reduction="mean")(y_pred, y_true)
        scad_reg = torch.sum(scad_penalty(self.linear.weight, self.alpha, self.a))

        loss = mse_loss + self.alpha * scad_reg

        return loss.float()

    def fit(self, X, y, num_epochs=100, learning_rate=0.01):
        """
        Fit the SCADRegressor model to the training data.

        Args:
            X (Tensor): Input data with shape (num_samples, input_size).
            y (Tensor): Target values with shape (num_samples, 1).
            num_epochs (int): Number of training epochs.
            learning_rate (float): Learning rate for optimization.
        """
        optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        print_interval = num_epochs // 10

        for epoch in range(num_epochs):
            self.train()
            optimizer.zero_grad()
            y_pred = self(X)
            loss = self.loss(y_pred, y)
            loss.backward()
            optimizer.step()

            if (epoch + 1) % print_interval == 0:
                print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}")

    def predict(self, X):
        """
        Predict target values for input data.

        Args:
            X (Tensor): Input data with shape (num_samples, input_size).

        Returns:
            Tensor: Predicted values with shape (num_samples, 1).
        """
        self.eval()
        with torch.no_grad():
            y_pred = self(X)
        return y_pred

    def get_coefficients(self):
        """
        Get the coefficients (weights) of the linear regression layer.

        Returns:
            Tensor: Coefficients with shape (output_size, input_size).
        """
        return self.linear.weight


def scad_penalty(beta_hat, lambda_val, a_val):
    is_linear = torch.abs(beta_hat) <= lambda_val
    is_quadratic = torch.logical_and(
        lambda_val < torch.abs(beta_hat), torch.abs(beta_hat) <= a_val * lambda_val
    )
    is_constant = (a_val * lambda_val) < torch.abs(beta_hat)

    linear_part = lambda_val * torch.abs(beta_hat) * is_linear
    quadratic_part = (
        (2 * a_val * lambda_val * torch.abs(beta_hat) - beta_hat**2 - lambda_val**2)
        / (2 * (a_val - 1))
        * is_quadratic
    )
    constant_part = (lambda_val**2 * (a_val + 1)) / 2 * is_constant
    return (linear_part + quadratic_part + constant_part).float()  # Cast to float

Testing Model on real dataset and determining which variables should be used


In [31]:
import torch
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load dataset
data = fetch_california_housing()
X, y = data["data"], data["target"]

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=dtype).float()
y_train_tensor = torch.tensor(y_train, dtype=dtype).float()

# Instantiate SCADRegressor
input_size = X_train.shape[1]
scad_model = SCADRegressor(input_size=input_size, alpha=1.0, a=3.7)

# Train the model
scad_model.fit(
    X_train_tensor, y_train_tensor.view(-1, 1), num_epochs=10000, learning_rate=0.01
)

# Get coefficients
coefficients = scad_model.get_coefficients()

# Analyze feature importance (i+1 so we do not say feature 0)
importance = torch.abs(coefficients)
print("Feature importance according to SCAD:")
for i, imp in enumerate(importance.squeeze()):
    print(f"Feature {i+1}: {imp.item()}")

# Perform variable selection based on importance (i+1 so we do not say feature 0)
selected_features = []
for i, imp in enumerate(importance.squeeze()):
    if imp.item() > 0.001:
        selected_features.append(i + 1)
print("Selected features based on SCAD importance:")
print(selected_features)

Epoch [1000/10000], Loss: 5.546572208404541
Epoch [2000/10000], Loss: 5.54582405090332
Epoch [3000/10000], Loss: 5.547928810119629
Epoch [4000/10000], Loss: 5.548254489898682
Epoch [5000/10000], Loss: 5.548460483551025
Epoch [6000/10000], Loss: 5.548340797424316
Epoch [7000/10000], Loss: 5.547878265380859
Epoch [8000/10000], Loss: 5.54454231262207
Epoch [9000/10000], Loss: 5.554252624511719
Epoch [10000/10000], Loss: 5.5503716468811035
Feature importance according to SCAD:
Feature 1: 0.3019988536834717
Feature 2: 0.0023206511978060007
Feature 3: 0.0014103325083851814
Feature 4: 0.0006426338804885745
Feature 5: 0.0007898104377090931
Feature 6: 0.0029451940208673477
Feature 7: 0.0004759312141686678
Feature 8: 0.0006278303917497396
Selected features based on SCAD importance:
[1, 2, 3, 6]


##500 data sets where the input features have a strong correlation structure (0.9) and apply ElasticNet, SqrtLasso and SCAD to check which method produces the best approximation of an ideal solution, such as a "betastar" you design with a sparsity pattern.


ElasticNet Regressor


In [32]:
class ElasticNet(nn.Module):
    def __init__(self, input_size, alpha=1.0, l1_ratio=0.5):
        """
        Initialize the ElasticNet regression model.

        Args:
            input_size (int): Number of input features.
            alpha (float): Regularization strength. Higher values of alpha
                emphasize L1 regularization, while lower values emphasize L2 regularization.
            l1_ratio (float): The ratio of L1 regularization to the total
                regularization (L1 + L2). It should be between 0 and 1.

        """
        super(ElasticNet, self).__init__()
        self.input_size = input_size
        self.alpha = alpha
        self.l1_ratio = l1_ratio

        # Define the linear regression layer
        self.linear = nn.Linear(input_size, 1, bias=False, device=device, dtype=dtype)

    def forward(self, x):
        """
        Forward pass of the ElasticNet model.

        Args:
            x (Tensor): Input data with shape (batch_size, input_size).

        Returns:
            Tensor: Predicted values with shape (batch_size, 1).

        """
        return self.linear(x)

    def loss(self, y_pred, y_true):
        """
        Compute the ElasticNet loss function.

        Args:
            y_pred (Tensor): Predicted values with shape (batch_size, 1).
            y_true (Tensor): True target values with shape (batch_size, 1).

        Returns:
            Tensor: The ElasticNet loss.

        """
        mse_loss = nn.MSELoss(reduction="mean")(y_pred, y_true)
        l1_reg = torch.norm(self.linear.weight, p=1)
        l2_reg = torch.norm(self.linear.weight, p=2)

        objective = (1 / 2) * mse_loss + self.alpha * (
            self.l1_ratio * l1_reg + (1 - self.l1_ratio) * (1 / 2) * l2_reg**2
        )

        return objective

    def fit(self, X, y, num_epochs=100, learning_rate=0.01):
        """
        Fit the ElasticNet model to the training data.

        Args:
            X (Tensor): Input data with shape (num_samples, input_size).
            y (Tensor): Target values with shape (num_samples, 1).
            num_epochs (int): Number of training epochs.
            learning_rate (float): Learning rate for optimization.

        """
        optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        print_interval = num_epochs // 10

        for epoch in range(num_epochs):
            self.train()
            optimizer.zero_grad()
            y_pred = self(X)  # A forward propagation
            loss = self.loss(y_pred, y)
            loss.backward()  # The optimizer computes the gradient
            optimizer.step()  # Here the optimizer updates the weights according to the chosen heuristics (Adam in this case)

            if (epoch + 1) % print_interval == 0:
                print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}")

    def predict(self, X):
        """
        Predict target values for input data.

        Args:
            X (Tensor): Input data with shape (num_samples, input_size).

        Returns:
            Tensor: Predicted values with shape (num_samples, 1).

        """
        self.eval()
        with torch.no_grad():
            y_pred = self(X)
        return y_pred

    def get_coefficients(self):
        """
        Get the coefficients (weights) of the linear regression layer.

        Returns:
            Tensor: Coefficients with shape (output_size, input_size).

        """
        return self.linear.weight

Square Root Lasso Regressor


In [33]:
class SqrtLasso(nn.Module):
    def __init__(self, input_size, alpha=0.1):
        """
        Initialize the  regression model.


        """
        super(SqrtLasso, self).__init__()
        self.input_size = input_size
        self.alpha = alpha

        # Define the linear regression layer
        self.linear = nn.Linear(input_size, 1, bias=False, device=device, dtype=dtype)

    def forward(self, x):
        """
        Forward pass of the model.

        Args:
            x (Tensor): Input data with shape (batch_size, input_size).

        Returns:
            Tensor: Predicted values with shape (batch_size, 1).

        """
        return self.linear(x)

    def loss(self, y_pred, y_true):
        """
        Compute the loss function.

        Args:
            y_pred (Tensor): Predicted values with shape (batch_size, 1).
            y_true (Tensor): True target values with shape (batch_size, 1).

        Returns:
            Tensor: The loss.

        """
        mse_loss = nn.MSELoss(reduction="mean")(y_pred, y_true)
        l1_reg = torch.norm(self.linear.weight, p=1, dtype=torch.float64)
        # l2_reg = torch.norm(self.linear.weight, p=2,dtype=torch.float64)

        loss = torch.sqrt(mse_loss) + self.alpha * (l1_reg)

        return loss

    def fit(self, X, y, num_epochs=200, learning_rate=0.01):
        """
        Fit the model to the training data.

        Args:
            X (Tensor): Input data with shape (num_samples, input_size).
            y (Tensor): Target values with shape (num_samples, 1).
            num_epochs (int): Number of training epochs.
            learning_rate (float): Learning rate for optimization.

        """
        optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        print_interval = num_epochs // 10

        for epoch in range(num_epochs):
            self.train()
            optimizer.zero_grad()
            y_pred = self(X)
            loss = self.loss(y_pred, y)
            loss.backward()
            optimizer.step()

            if (epoch + 1) % print_interval == 0:
                print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}")

    def predict(self, X):
        """
        Predict target values for input data.

        Args:
            X (Tensor): Input data with shape (num_samples, input_size).

        Returns:
            Tensor: Predicted values with shape (num_samples, 1).

        """
        self.eval()
        with torch.no_grad():
            y_pred = self(X)
        return y_pred

    def get_coefficients(self):
        """
        Get the coefficients (weights) of the linear regression layer.

        Returns:
            Tensor: Coefficients with shape (output_size, input_size).

        """
        return self.linear.weight

Generating Data


In [34]:
# we want to define a function for generating x with a prescribed number of obsvervations, features and Toeplitz correlation structure.
def make_correlated_features(num_samples, p, rho):
    vcor = []
    for i in range(p):
        vcor.append(rho**i)
    r = toeplitz(vcor)
    mu = np.repeat(0, p)
    x = np.random.multivariate_normal(mu, r, size=num_samples)
    return x

In [35]:
rho = 0.9
p = 10
n = 500
vcor = []
for i in range(p):
    vcor.append(rho**i)

In [36]:
x = make_correlated_features(n, p, rho)

In [37]:
beta = np.array([0, -1, 2, 3, 0, 0, 2])
beta = beta.reshape(-1, 1)
betastar = np.concatenate([beta, np.repeat(0, p - len(beta)).reshape(-1, 1)], axis=0)

In [38]:
# generate response with noise
y = x @ betastar + 1.5 * np.random.normal(size=(n, 1))

In [39]:
# put your data in torch tensors
x_torch = torch.tensor(x, device=device)
y_torch = torch.tensor(y, device=device)

In [40]:
ScadModel = SCADRegressor(input_size=x_torch.shape[1], a=0.5)

In [41]:
ScadModel.fit(x_torch.float(), y_torch.float(), num_epochs=10000, learning_rate=0.01)

Epoch [1000/10000], Loss: 6.28976583480835
Epoch [2000/10000], Loss: 4.741990566253662
Epoch [3000/10000], Loss: 4.741029739379883
Epoch [4000/10000], Loss: 4.738697528839111
Epoch [5000/10000], Loss: 4.741764545440674
Epoch [6000/10000], Loss: 4.742648124694824
Epoch [7000/10000], Loss: 4.740178108215332
Epoch [8000/10000], Loss: 4.7432026863098145
Epoch [9000/10000], Loss: 4.735722541809082
Epoch [10000/10000], Loss: 4.740964889526367


In [42]:
beta_scad = ScadModel.get_coefficients()[0].cpu().detach().numpy()

In [43]:
ElasticNetModel = ElasticNet(input_size=x.shape[1], alpha=0.01, l1_ratio=0.5)
ElasticNetModel.fit(x_torch, y_torch, num_epochs=10000, learning_rate=0.01)

Epoch [1000/10000], Loss: 1.2737965594525344
Epoch [2000/10000], Loss: 1.2215088904207079
Epoch [3000/10000], Loss: 1.2200256873241997
Epoch [4000/10000], Loss: 1.2200088592006804
Epoch [5000/10000], Loss: 1.220009168367676
Epoch [6000/10000], Loss: 1.220010815354122
Epoch [7000/10000], Loss: 1.2200119177049742
Epoch [8000/10000], Loss: 1.2200100202660544
Epoch [9000/10000], Loss: 1.2200127827163825
Epoch [10000/10000], Loss: 1.2200155689322731


In [44]:
beta_ElasticNet = ElasticNetModel.get_coefficients()[0].cpu().detach().numpy()

In [45]:
SqrtLassoModel = SqrtLasso(input_size=x.shape[1], alpha=0.1)
SqrtLassoModel.fit(x_torch, y_torch, num_epochs=10000, learning_rate=0.01)

Epoch [1000/10000], Loss: 2.156723531032999
Epoch [2000/10000], Loss: 2.1530775725202758
Epoch [3000/10000], Loss: 2.1530040856070323
Epoch [4000/10000], Loss: 2.153180355991468
Epoch [5000/10000], Loss: 2.1532568680831883
Epoch [6000/10000], Loss: 2.1532440442801635
Epoch [7000/10000], Loss: 2.1533218700612156
Epoch [8000/10000], Loss: 2.153372707842402
Epoch [9000/10000], Loss: 2.1534436188598463
Epoch [10000/10000], Loss: 2.1533077547257866


In [46]:
beta_SqrtLassoModel = SqrtLassoModel.get_coefficients()[0].cpu().detach().numpy()

In [47]:
rounded_beta_scad = np.round(beta_scad, decimals=4)
rounded_beta_ElasticNet = np.round(beta_ElasticNet, decimals=4)
rounded_beta_SqrtLassoModel = np.round(beta_SqrtLassoModel, decimals=4)

In [48]:
stacked_array = np.column_stack(
    [betastar, rounded_beta_scad, rounded_beta_ElasticNet, rounded_beta_SqrtLassoModel]
)



# Convert to DataFrame


df = pd.DataFrame(
    stacked_array,
    columns=["betastar", "Beta SCAD", "Beta ElasticNet", "Beta SquareRootLasso"],
)


df

,betastar,Beta SCAD,Beta ElasticNet,Beta SquareRootLasso
0,0.0,0.0015,0.0007,-0.0007
1,-1.0,-0.0028,-0.8188,-0.0022
2,2.0,1.0779,1.8418,0.9981
3,3.0,2.9012,2.5997,2.7448
4,0.0,-0.0009,0.1915,0.0495
5,0.0,-0.0022,0.3135,0.2795
6,2.0,1.9663,1.4912,1.5254
7,0.0,-0.0014,0.1049,0.1281
8,0.0,0.0005,0.1014,0.0489
9,0.0,0.0018,0.0580,0.0123


Comparing MSE's


In [49]:
print("SCAD MSE:", mse(y, x @ beta_scad))
print("ElasticNet MSE:", mse(y, x @ beta_ElasticNet))
print("Square Root Lasso MSE:", mse(y, x @ beta_SqrtLassoModel))

SCAD MSE: 2.4807817920735182
ElasticNet MSE: 2.298798106960634
Square Root Lasso MSE: 2.47832341461302


https://akhipath03.github.io/AdvMachineLearning/HW1/
